In [8]:
from pathlib import Path
print(Path.cwd())

c:\Users\ersan\OneDrive\Masaüstü\ders\ceng481\NBA-Game-Prediction-using-Artificial-Neural-Networks\notebooks


In [9]:
import requests
import re
from bs4 import BeautifulSoup
from datetime import datetime
from pathlib import Path

import pdfplumber
import pandas as pd


def download_latest_injury_pdf(base_url: str, save_dir: Path) -> Path | None:
    """
    NBA official injury report sayfasındaki en güncel PDF'i bulur ve indirir.
    Returns: İndirilen PDF yolunu döner veya None
    """
    headers = {"User-Agent": "Mozilla/5.0"}

    resp = requests.get(base_url, headers=headers)
    if resp.status_code != 200:
        print(f"❌ Sayfa alınamadı: {resp.status_code}")
        return None

    soup = BeautifulSoup(resp.content, "html.parser")
    links = soup.select("a[href*='Injury-Report_']")

    if not links:
        print("⚠️ Hiç PDF linki bulunamadı.")
        return None

    pdf_links = []
    for link in links:
        href = link.get("href")
        if href and "Injury-Report_" in href and href.endswith(".pdf"):
            full_url = href if href.startswith("http") else f"https://ak-static.cms.nba.com{href}"
            timestamp = full_url.split("Injury-Report_")[1].replace(".pdf", "")
            pdf_links.append((timestamp, full_url))

    def parse_ts(ts: str):
        try:
            return datetime.strptime(ts, "%Y-%m-%d_%I%M%p")
        except Exception:
            return datetime.min

    pdf_links.sort(key=lambda x: parse_ts(x[0]), reverse=True)
    latest_ts, latest_url = pdf_links[0]

    save_dir.mkdir(parents=True, exist_ok=True)
    filename = f"Injury-Report_{latest_ts}.pdf"
    save_path = save_dir / filename

    r = requests.get(latest_url, headers=headers)
    r.raise_for_status()
    save_path.write_bytes(r.content)

    print(f"📄 En güncel PDF kaydedildi: {save_path}")
    return save_path

In [10]:
from pathlib import Path

download_latest_injury_pdf(
    base_url="https://official.nba.com/nba-injury-report-2025-26-season/",
    save_dir=Path("../data_raw/injury_reports_raw")
)
#bu çalıştırılınca pdf'i indiriyor

📄 En güncel PDF kaydedildi: ..\data_raw\injury_reports_raw\Injury-Report_2025-12-19_12AM.pdf


WindowsPath('../data_raw/injury_reports_raw/Injury-Report_2025-12-19_12AM.pdf')

In [11]:
import re
from pathlib import Path
import pandas as pd
import pdfplumber

# --- NEW: canonical team mapping helpers ---
NBA_TEAMS_CANONICAL = [
    "Atlanta Hawks", "Boston Celtics", "Brooklyn Nets", "Charlotte Hornets",
    "Chicago Bulls", "Cleveland Cavaliers", "Dallas Mavericks", "Denver Nuggets",
    "Detroit Pistons", "Golden State Warriors", "Houston Rockets", "Indiana Pacers",
    "LA Clippers", "Los Angeles Lakers", "Memphis Grizzlies", "Miami Heat",
    "Milwaukee Bucks", "Minnesota Timberwolves", "New Orleans Pelicans",
    "New York Knicks", "Oklahoma City Thunder", "Orlando Magic",
    "Philadelphia 76ers", "Phoenix Suns", "Portland Trail Blazers",
    "Sacramento Kings", "San Antonio Spurs", "Toronto Raptors",
    "Utah Jazz", "Washington Wizards",
]

def _norm_team(s: str) -> str:
    # keep only letters+digits, lowercase (spaces/punct removed)
    return re.sub(r"[^a-z0-9]+", "", (s or "").lower())

_CANON_TEAM_BY_NORM = {_norm_team(t): t for t in NBA_TEAMS_CANONICAL}

def _fix_team_spacing(team: str) -> str:
    """
    Fixes spacing issues and maps to canonical NBA team names when possible.
    Examples:
      MiamiHeat -> Miami Heat
      OklahomaCityThunder -> Oklahoma City Thunder
      LAClippers -> LA Clippers
    """
    if not team:
        return team

    team = team.strip()

    # First: add spaces for CamelCase-ish strings
    team = re.sub(r"(?<=[a-z])(?=[A-Z])", " ", team)
    team = re.sub(r"\s+", " ", team).strip()

    # Then: canonicalize by normalized match (very robust against missing spaces)
    norm = _norm_team(team)
    if norm in _CANON_TEAM_BY_NORM:
        return _CANON_TEAM_BY_NORM[norm]

    # Sometimes there can be extra tokens; try best-match by containment (rare but helps)
    # pick the longest canonical normalized string that appears in norm
    best = None
    for cn_norm, cn in _CANON_TEAM_BY_NORM.items():
        if cn_norm and cn_norm in norm:
            if best is None or len(cn_norm) > len(_norm_team(best)):
                best = cn
    return best if best else team


def parse_injury_pdf_text(pdf_path: Path, csv_path: Path | None = None) -> pd.DataFrame | None:
    """
    Parses NBA injury report PDF (table-like text) into rows:
    report_date, game_date, game_time, matchup, team, player_name, status, reason
    Writes to injuries_latest.csv by default (same folder).
    """
    if not pdf_path.exists():
        print(f"❌ PDF bulunamadı: {pdf_path}")
        return None

    # report_date from filename: Injury-Report_YYYY-MM-DD_....
    m = re.search(r"Injury-Report_(\d{4}-\d{2}-\d{2})_", pdf_path.name)
    report_date = m.group(1) if m else None
    default_game_date = pd.to_datetime(report_date, errors="coerce").strftime("%m/%d/%Y") if report_date else None

    STATUS = ("Out", "Questionable", "Probable", "Available", "Doubtful")

    # state for table continuation lines
    current_game_date = default_game_date
    current_game_time = None
    current_matchup = None
    current_team = None

    rows = []
    last_row_idx = None  # for reason continuation

    # helpers
    def has_status(line: str) -> bool:
        return any(f" {s} " in f" {line} " for s in STATUS)

    PLAYER_RE = r"[A-Za-zÀ-ÖØ-öø-ÿ'.-]+,\s*[A-Za-zÀ-ÖØ-öø-ÿ'.-]+"

    def extract_player_status_reason(tail: str):
        """
        tail: starts with 'Lastname, Firstname ...'
        returns (player_name, status, reason)
        """
        mps = re.search(
            rf"(?P<player>{PLAYER_RE})\s+"
            r"(?P<status>Out|Questionable|Probable|Available|Doubtful)\b"
            r"(?P<reason>.*)$",
            tail.strip()
        )
        if not mps:
            return None, None, None
        player = mps.group("player").strip()
        status = mps.group("status").strip()
        reason = mps.group("reason").strip(" -–;:")
        return player, status, reason

    def add_row(player, status, reason):
        nonlocal last_row_idx
        rows.append({
            "report_date": report_date,
            "game_date": current_game_date,
            "game_time": current_game_time,
            "matchup": current_matchup,
            "team": current_team,
            "player_name": player,
            "status": status,
            "reason": reason
        })
        last_row_idx = len(rows) - 1

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if not text:
                continue

            for raw in text.split("\n"):
                line = raw.strip()
                if not line:
                    continue

                # skip headers/footers
                if line.startswith("Injury Report:") or line.startswith("Page "):
                    continue
                if line.startswith("Game Date") and "Matchup" in line:
                    continue

                # skip not submitted
                if "NOT YET SUBMITTED" in line:
                    continue

                # CASE 1: full row with explicit game date
                mfull = re.match(
                    r"^(?P<gdate>\d{1,2}/\d{1,2}/\d{4})\s+"
                    r"(?P<gtime>\d{1,2}:\d{2})\s*\(ET\)\s+"
                    r"(?P<matchup>[A-Z]{2,3}@\s*[A-Z]{2,3}|[A-Z]{2,3}@[A-Z]{2,3})\s+"
                    r"(?P<rest>.+)$",
                    line
                )
                if mfull:
                    current_game_date = mfull.group("gdate")
                    current_game_time = f"{mfull.group('gtime')}(ET)"
                    current_matchup = re.sub(r"\s+", "", mfull.group("matchup"))
                    rest = mfull.group("rest").strip()

                    mp = re.search(PLAYER_RE, rest)
                    if not mp:
                        continue

                    team_part = rest[:mp.start()].strip()
                    tail = rest[mp.start():].strip()

                    current_team = _fix_team_spacing(team_part)

                    player, status, reason = extract_player_status_reason(tail)
                    if player and status:
                        add_row(player, status, reason)
                    continue

                # CASE 2: row without date but with time (same report date)
                mtime = re.match(
                    r"^(?P<gtime>\d{1,2}:\d{2})\s*\(ET\)\s+"
                    r"(?P<matchup>[A-Z]{2,3}@\s*[A-Z]{2,3}|[A-Z]{2,3}@[A-Z]{2,3})\s+"
                    r"(?P<rest>.+)$",
                    line
                )
                if mtime:
                    current_game_date = current_game_date or default_game_date
                    current_game_time = f"{mtime.group('gtime')}(ET)"
                    current_matchup = re.sub(r"\s+", "", mtime.group("matchup"))
                    rest = mtime.group("rest").strip()

                    mp = re.search(PLAYER_RE, rest)
                    if not mp:
                        continue
                    team_part = rest[:mp.start()].strip()
                    tail = rest[mp.start():].strip()

                    current_team = _fix_team_spacing(team_part)

                    player, status, reason = extract_player_status_reason(tail)
                    if player and status:
                        add_row(player, status, reason)
                    continue

                # --- NEW CASE 3: line contains player+status but starts with TEAM or MATCHUP+TEAM ---
                # Examples from PDF:
                #   "Charlotte Hornets Ball, LaMelo Questionable ..."
                #   "NYK@IND New York Knicks Anunoby, OG Questionable ..."
                if has_status(line) and re.search(PLAYER_RE, line):
                    mp = re.search(PLAYER_RE, line)
                    prefix = line[:mp.start()].strip()
                    tail = line[mp.start():].strip()

                    if prefix:
                        # prefix may be "TEAM ..." or "MATCHUP TEAM ..."
                        mm = re.match(r"^(?P<matchup>[A-Z]{2,3}@\s*[A-Z]{2,3}|[A-Z]{2,3}@[A-Z]{2,3})\s+(?P<team>.+)$", prefix)
                        if mm:
                            current_matchup = re.sub(r"\s+", "", mm.group("matchup"))
                            team_part = mm.group("team").strip()
                        else:
                            team_part = prefix

                        current_team = _fix_team_spacing(team_part)

                    # If prefix empty, it's a continuation line: use existing current_team
                    if current_team is None:
                        continue

                    player, status, reason = extract_player_status_reason(tail)
                    if player and status:
                        add_row(player, status, reason)
                    continue

                # CASE 4: reason wrapped to next line(s)
                if last_row_idx is not None and (
                    line.startswith("Injury/Illness") or line.startswith("G League") or line.startswith("Not With Team")
                ):
                    prev = rows[last_row_idx]["reason"]
                    rows[last_row_idx]["reason"] = (prev + " " + line).strip() if prev else line
                    continue

    df = pd.DataFrame(rows)

    if csv_path is None:
        csv_path = pdf_path.parent / "injuries_latest.csv"

    df.to_csv(csv_path, index=False)
    print(f"✅ Injury CSV kaydedildi: {csv_path} ({len(df)} satır)")
    return df


In [12]:
pdf_path = download_latest_injury_pdf(
    "https://official.nba.com/nba-injury-report-2025-26-season/",
    Path("../data_raw/injury_reports_raw")
)

if pdf_path:
    parse_injury_pdf_text(pdf_path)

📄 En güncel PDF kaydedildi: ..\data_raw\injury_reports_raw\Injury-Report_2025-12-19_12AM.pdf
✅ Injury CSV kaydedildi: ..\data_raw\injury_reports_raw\injuries_latest.csv (31 satır)
